In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [2]:
def generate_equal_classes(n_full, num_classes):
    if n_full % num_classes != 0:
        raise ValueError("n_full is not evenly divisible by num_classes")
    
    # Number of instances per class
    n_per_class = n_full // num_classes
    
    # Generate and shuffle array
    classes = np.concatenate([np.full(n_per_class, i) for i in range(num_classes)])
    np.random.shuffle(classes)
    
    return classes.tolist()

In [3]:
def count_memorized_points(X, y):
    knn = KNeighborsClassifier(n_neighbors=1)
    n_memorized = 0
    n_full = len(y)
    for i in range(n_full):
        # Delete the row and y
        X_train = np.delete(X, i, axis=0)
        y_train = np.delete(y, i)
        knn.fit(X_train, y_train)
        # Test if deleted y is still predicted correctly
        if knn.predict([X[i]]) == y[i]:
            # If y is predicted correctly, despite dropping the row, then that row was redundant
            n_memorized += 1 # Increment the count of redundant training points

    return n_full - n_memorized # removing the redundant points, leaves the kept 'necessary' rows

In [4]:
# Here I use Alg 8 to count thresholds and test accuracy
# Note this is purely out of curiosity and not nessicary for 6.1
def count_thresholds(X, y):
    # Note that Alg 8 assumes a perfect ML model
    thresholds = 0

    sum_table = [(sum(row), label) for row, label in zip(X, y)] # Sum x rows
    sorted_table = sorted(sum_table, key=lambda x: x[0]) # Sort by increasing x

    needed_x = []
    needed_y = []

    current_class = sorted_table[0][1] # Start by setting current_class, to the class of the first row
    for row, row_class in sorted_table:
        # If the sign changes, then that row is 'necessary', hence thresholds is incremented
        if row_class != current_class:
            current_class = row_class
            thresholds += 1
        # else: # Doing it this way gives better results, probably because its including more rows
            needed_x.append([row]) 
            needed_y.append(row_class)
             
    sorted_x, sorted_y = zip(*sorted_table)
    sorted_x_arr = []

    for row in sorted_x:
        sorted_x_arr.append([row])
    sorted_y = list(sorted_y) 

    # Now test the 'needed' rows against the entire sorted table
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(needed_x, needed_y)
    y_pred = knn.predict(sorted_x_arr) # test on full
    accuracy = accuracy_score(sorted_y, y_pred)
    # print(f'    Accuracy = {accuracy}, y_needed = {needed_y}')
    
    return thresholds, sorted_x_arr, sorted_y, accuracy

In [5]:
def run_experiment(D: int, n_full: int, num_classes):
    n_mem_count = 0
    n_mem_count_1D = 0
    n_thresholds = 0
    n_alg_8_acc = 0
    
    n_experiments = 50
    for i in range(n_experiments):
        X = np.random.rand(n_full, D)
        
        # I tested with both random y and y with equal number of 1's and 0's
        # y = np.random.randint(0, 2, size=n_full)
        y = generate_equal_classes(n_full, num_classes)

        thresholds, sorted_x, sorted_y, alg_8_acc = count_thresholds(X, y)
        n_thresholds += thresholds
        n_alg_8_acc += alg_8_acc

        n_mem_count_1D += count_memorized_points(sorted_x, sorted_y)
        n_mem_count += count_memorized_points(X, y)

    # Calculate all the averages
    n_avg_1D = n_mem_count_1D/n_experiments
    n_avg = n_mem_count/n_experiments
    avg_thresholds = n_thresholds/n_experiments
    avg_alg_8_acc = n_alg_8_acc/n_experiments

    base = f"Base Results: d={D}: n_full={n_full}, Avg. req. points for memorization n_avg={n_avg:.2f}, n_full/n_avg={n_full/n_avg}"
    row_sum = f"Rows Sum Results: d={D}: n_full={n_full}, Avg. req. points for memorization n_avg={n_avg_1D:.2f}, n_full/n_avg={n_full/n_avg_1D} "
    thresh = f"Threshold Results: d={D}: n_full={n_full}, Avg. Thresh={avg_thresholds}, Avg. Alg_8 Accuracy={avg_alg_8_acc}, n_full/thresh_avg={n_full/avg_thresholds}"
    return base, row_sum, thresh

### Results:
**Only the Base Results are required for 6.1**  
The additional results in _Findings from the Algorithm 8_ (Rows Sum Result and Threshold Results) were merely for my own exploration of Algorithm 8 on page 118. 



In [6]:
np.random.seed(64)
# All experiments are being tested with 4 classes
# Since 6.2 is a multi-class dataset, I use num_classes = 4
results = {}
results['2D'] = run_experiment(D=2, n_full=4, num_classes=4)
print(results['2D'][0])

results['4D'] = run_experiment(D=4, n_full=16, num_classes=4)
print(results['4D'][0])

results['8D'] = run_experiment(D=8, n_full=256, num_classes=4)
print(results['8D'][0])

results['16D'] = run_experiment(D=16, n_full=1024, num_classes=4)
print(results['16D'][0])

Base Results: d=2: n_full=4, Avg. req. points for memorization n_avg=4.00, n_full/n_avg=1.0
Base Results: d=4: n_full=16, Avg. req. points for memorization n_avg=13.08, n_full/n_avg=1.2232415902140672
Base Results: d=8: n_full=256, Avg. req. points for memorization n_avg=193.20, n_full/n_avg=1.3250517598343685
Base Results: d=16: n_full=1024, Avg. req. points for memorization n_avg=771.04, n_full/n_avg=1.3280763643909526




#### Examining the Base Results:
The base results are done with the _count_memorized_points_ function that drops rows that do not negatively impact accuracy.
You can see that as the dimensions (D) increase n_full/n_avg approaches 2.

- D=2, n_full/n_avg = 1.0
- D=4, n_full/n_avg = 1.2232415902140672
- D=8, n_full/n_avg = 1.3250517598343685
- D=16, n_full/n_avg = 1.3280763643909526

_Results are from code block above using rand seed of 64_

#### Theoretical Information Capacity:

_Definition 5.1_ Information Capacity of a linear Separator C = c/(c-1)
- 4 classes -> c = 4
- **C = 4/(4-1) = 1.3334**

Therefore our **experimental results** align with the **theoretical calculation**

Note: This theoretical result is also achieved using the threshold counting theorem form Algorithm 8 (see below)


#### Findings from the Algorithm 8:
Algorithm 8 requires two _data prep_ steps: 1. Summing the x rows. 2. Sorting the table by increasing x.  
Once the Data is prepared, then you iterate through the new table increment thresholds every time the class (y) changes.
- I wanted to test the _count_memorized_points_ function, on the prepared data table as outlined in Algorithm 8. The results of this test are the **'Rows Sum Results'**
- Below you can see that summing and sorting the data has _negligible effect_ on the n_full/n_avg.

In [7]:
# Print the Experimental Results with the summed and sorted table from Alg 8
print(results['2D'][1])
print(results['4D'][1])
print(results['8D'][1])
print(results['16D'][1])

Rows Sum Results: d=2: n_full=4, Avg. req. points for memorization n_avg=4.00, n_full/n_avg=1.0 
Rows Sum Results: d=4: n_full=16, Avg. req. points for memorization n_avg=13.08, n_full/n_avg=1.2232415902140672 
Rows Sum Results: d=8: n_full=256, Avg. req. points for memorization n_avg=193.12, n_full/n_avg=1.3256006628003314 
Rows Sum Results: d=16: n_full=1024, Avg. req. points for memorization n_avg=768.48, n_full/n_avg=1.3325005205080158 



#### Counting Thresholds
Continuing to implement the threshold increment part of Algorithm 8 I decided to test a KNN on only rows that increased the threshold:
- In this test I began to notice that the **average threshold count was getting very close to n_avg** as the D increased. 
- Therefore I decided to record n_full/thresh_avg, which stays around 1.334, regardless of the dimensions. 
    - Having n_full/thresh_avg ≈ 1.334 Demonstrates that the **Theoretical upper limit** for n_full/n_avg to be 1.334 for classification with 4 classes
- An additional interesting find was that **as D increases, the Accuracy** of training a KNN on **just** the threshold rows (Alg_8 Accuracy), increased with dimensions (D):
    - D=2, Avg. Alg_8 Accuracy = 0.75
    - D=4, Avg. Alg_8 Accuracy = 0.855
    - D=8, Avg. Alg_8 Accuracy = 0.873125
    - D=16 Abg. Alg_8 Accuracy = 0.87453125


_Alg_8 Accuracy values are displayed by the output of the codeblock below_





In [8]:
# Print the Results from counting thresholds with the summed and sorted table from Alg 8
print(results['2D'][2])
print(results['4D'][2])
print(results['8D'][2])
print(results['16D'][2])

Threshold Results: d=2: n_full=4, Avg. Thresh=3.0, Avg. Alg_8 Accuracy=0.75, n_full/thresh_avg=1.3333333333333333
Threshold Results: d=4: n_full=16, Avg. Thresh=12.32, Avg. Alg_8 Accuracy=0.855, n_full/thresh_avg=1.2987012987012987
Threshold Results: d=8: n_full=256, Avg. Thresh=192.44, Avg. Alg_8 Accuracy=0.873125, n_full/thresh_avg=1.3302847640823114
Threshold Results: d=16: n_full=1024, Avg. Thresh=767.68, Avg. Alg_8 Accuracy=0.87453125, n_full/thresh_avg=1.3338891204668613


### Conclusion:
In conclusion, I used KNN to Empirically show that the information capacity limit approaches 2 prediction bits per parameter as the dimensionality (D) Increases

To show this I did the following steps:
1. Make a table of d dimensions and n rows, populated the x with random noise and Y with **4 classes** of equal distribution.
2. Crated a _count_memorized_points_ function that iteratively drops rows from the table, and tests if the accuracy drops.
3. Did 50 experiment runs per D and averaged the _count_memorized_points_ results into n_avg. 
4. Test the following configurations (D=2, n_full=4), (D=4, n_full=16), (D=8, n_full=256)
5. Observed that as dimensionality (D) increases, n_full/n_avg approaches 2.



Finally out of my own curiosity I implemented parts of Algorithm 8 to enhance my understanding of the larger lesson  
Implementing Algorithm 8 showed:
 - That for Classification of 4 classes, the theoretical upper limit is n_full/n_avg ≈ 1.33 (This matches the Information Capacity C calculation)
 - Training on rows that increment the threshold increases accuracy with dimensionality